In [ ]:
# importing necessary standard libraries
import csv
import json
import xml.etree.ElementTree as ET

In [ ]:
#Read data from user_data.csv
with open('./files/user_data.csv') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    csv_data = [row for row in csv_reader]

In [ ]:
#Read data from user_data.json
with open('user_data.json') as json_file:
    json_data = json.load(json_file)

In [ ]:
#Read data from user_data.xml
import xml.etree.ElementTree as ET
xml_data = []
tree = ET.parse('./files/user_data.xml')
root = tree.getroot()
for child in root:
    record = {}
    for item in child.attrib.items():
        record[item[0]] = item[1]
    xml_data.append(record)

In [ ]:
import re

with open("./files/user_data.txt", "r") as file:
    text_data = file.read()

customer_names = re.findall(r"[A-Z][a-z]+ [A-Z][a-z]+", text_data)
print(customer_names)


The data in the text file is not in a structured format like the data in the CSV, JSON and XML files. It appears to be unstructured text data that contains different types of messages, such as customer complaints, employee promotions and birthday wishes. These messages do not contain any information that can be directly combined with the data from the other files.
It's unclear how this text data is related to the structured data you've provided earlier. If you want to combine this text data with the structured data, you would need to extract meaningful information from these messages and map it to the appropriate fields in the structured data.

Some of the customer names are mentioned in the text data you provided. For example, "Shane Chambers" and "Suzanne Wright" are mentioned in the text data. Depending on the format of the text data, you might be able to extract customer names using regular expressions or text processing techniques.
If the format of the text data is consistent and customer names are always mentioned in a certain format, you could extract the customer names using regular expressions or string manipulation.
However, if the format of the text data is inconsistent or customer names are not mentioned in a consistent format, it might be more challenging to extract customer names.
It's also possible that the text data is not related to the structured data, and the customer names are not mentioned in the text data.

## So at the end, I concluded not to add the data from the user_data.txt because it just contained the un-relative data to what was present in the other files.

In [ ]:
# combining data from multiple sources into a single cohesive record for each customer, allowing for easier analysis and management of the data.
combined_data = []
for csv_record in csv_data:
    for json_record in json_data:
        if csv_record['First Name'] == json_record['firstName'] and csv_record['Second Name'] == json_record['lastName']:
            combined_record = csv_record.copy()
            combined_record.update(json_record)
            for xml_record in xml_data:
                if csv_record['First Name'] == xml_record['firstName'] and csv_record['Second Name'] == xml_record['lastName']:
                    combined_record.update(xml_record)
            combined_data.append(combined_record)

In [ ]:
# deleting inconsistent and duplicate data
for record in combined_data:
    del record['firstName']
    del record['lastName']
    del record['sex']
    del record['age']

In [ ]:
combined_data

In [ ]:
# install Pony
!pip install pony

In [ ]:
!pip install pymysql

In [ ]:
# Importing the pony.orm library
from pony.orm import *
# Initializing the database object
db = Database()

In [ ]:
# Define the CombinedData class as an entity in the database
class Student_bi25yy(db.Entity):
  # # Define required fields for the CombinedData entity
    firstName = Required(str)
    lastName = Required(str)
    age = Required(int)
    vehicleMake = Required(str)
    vehicleModel = Required(str)
    vehicleYear = Required(str)
    vehicleType = Required(str)
    iban = Required(str)
    creditCardNumber = Required(str)
    creditCardSecurityCode = Required(str)
    creditCardStartDate = Required(str)
    creditCardEndDate = Required(str)
    addressMain = Required(str)
    addressCity = Required(str)
    addressPostcode = Required(str)
    retired = Required(bool)
    dependants = Required(int)
    maritalStatus = Required(str)
    salary = Optional(int)
    pension = Optional(int)
    company = Optional(str)
    commuteDistance = Optional(float)
# The above code defines the class CombinedData as an ORM entity that can be used to interact with the database.
# The fields defined in the class are used to define the table and the columns in the database.
# The fields are defined as required or optional, which is used to enforce the validation of the data that is stored in the table.
# The class is also used to define the data type of each column, which is used to enforce the validation of the data that is stored in the table.

In [ ]:
# This code binds the database to the sqlite provider, specifying the filename and location for the database file.
# The 'create_db' parameter is set to True to create the database file if it does not already exist.
# The generate_mapping function is then called, with the create_tables parameter set to True to create the tables in the database if they do not already exist.
db.bind(provider='mysql', host='europa.ashley.work', user='student_bi25yy', passwd='iE93F2@8EhM@1zhD&u9M@K', db='student_bi25yy')
db.generate_mapping(create_tables=True)

In [ ]:
with db_session:
  for record in combined_data:
    try:
      Student_bi25yy(firstName = record['First Name'],
                   lastName = record['Second Name'],
                   age = int(record['Age (Years)']),
                   vehicleMake = record['Vehicle Make'],
                   vehicleModel = record['Vehicle Model'],
                   vehicleYear = record['Vehicle Year'],
                   vehicleType = record['Vehicle Type'],
                   iban = record['iban'],
                   creditCardNumber = record['credit_card_number'],
                   creditCardSecurityCode = record['credit_card_security_code'],
                   creditCardStartDate = record['credit_card_start_date'],
                   creditCardEndDate = record['credit_card_end_date'],
                   addressMain = record['address_main'],
                   addressCity = record['address_city'],
                   addressPostcode = record['address_postcode'],
                   retired = record['retired'],
                   dependants = int(record['dependants'])
                   if record['dependants']
                   else None,
                   maritalStatus = record['marital_status'],
                   salary = int(record['salary'])
                   if record['salary']
                   else None,
                   pension = int(record['pension'])
                   if record['pension']
                   else None,
                   company = record['company'],
                   commuteDistance = float(record['commute_distance'])
                   if record['commute_distance']
                   else None)
    except ValueError:
      print(f"Invalid value for record: {record}")

In [ ]:
with db_session:
    # retrieve the first 5 records in the CombinedData table
    records = select(r for r in Student_bi25yy)[:5]
    for record in records:
        print(record.to_dict())

In [ ]:
commit()